### Create worm datasets of various sizes by subsampling a larger combined datasets
---
*Last updated: 14 November 2023*

In [1]:
import os
import pickle

from pprint import PrettyPrinter
from itertools import combinations
from utils import init_random_seeds
from data._utils import generate_all_subsets, generate_subsets_of_size

CUDA device found.


In [9]:
# Set random seed for reproducibility
seed = 42
init_random_seeds(seed)

# Settings for pretty printing
pp = PrettyPrinter(indent=4, width=100, compact=True)

# Load combined dataset and print some information about it
# datadir = "custom"
datadir = "processed/neural"
# file = os.path.join(datadir, "combined_dataset.pickle")
file = os.path.join(datadir, "VanDerPol0000.pickle")
combined_dataset = pickle.load(open(file, "rb"))  # a few GBs large
num_worms = len(combined_dataset.keys())

In [10]:
# Print some information about the dataset
print(f"number of worms (N): {num_worms}", end="\n\n")
pp.pprint("all worm IDs: {}".format(list(combined_dataset.keys())))
print()
print(f"data keys for each worm' data:")
pp.pprint(list(combined_dataset["worm0"].keys()))
print()

# Example of choosing all subsets of size N-1 of the wormIDs when N
all_worm_ids = list(combined_dataset.keys())
worm_subsets = combinations(all_worm_ids, len(all_worm_ids) - 1)
print("Last five items of the first three subset sizes:")
for i, subset in enumerate(worm_subsets):
    print(f"\t{i}: {subset[-5:]}")
    if i == 2:
        break
print()

number of worms (N): 6

"all worm IDs: ['worm0', 'worm1', 'worm2', 'worm3', 'worm4', 'worm5']"

data keys for each worm' data:
[   'dataset', 'smooth_method', 'worm', 'calcium_data', 'smooth_calcium_data', 'residual_calcium',
    'smooth_residual_calcium', 'max_timesteps', 'time_in_seconds', 'dt', 'resample_median_dt',
    'num_neurons', 'num_named_neurons', 'num_unknown_neurons', 'named_neurons_mask',
    'unknown_neurons_mask', 'neurons_mask', 'slot_to_named_neuron', 'named_neuron_to_slot',
    'slot_to_unknown_neuron', 'unknown_neuron_to_slot', 'slot_to_neuron', 'neuron_to_slot',
    'original_time_in_seconds', 'original_dt', 'original_median_dt', 'original_calcium_data',
    'original_smooth_calcium_data', 'original_residual_calcium',
    'original_smooth_residual_calcium']

Last five items of the first three subset sizes:
	0: ('worm0', 'worm1', 'worm2', 'worm3', 'worm4')
	1: ('worm0', 'worm1', 'worm2', 'worm3', 'worm5')
	2: ('worm0', 'worm1', 'worm2', 'worm4', 'worm5')



In [11]:
# Generate 5 examples of all the possible size-3 subsets
# returns list of datasets containing 3 worms
subsets_3 = generate_subsets_of_size(
    combined_dataset,
    subset_size=3,
    max_subsets=5,
    as_assignment=True,
)
print(f"\ttype(subsets_3): {type(subsets_3)}")
print(f"\tlen(subsets_3): {len(subsets_3)}")
print(f"\ttype(subsets_3[0]): {type(subsets_3[0])}")
print()
for _, subset in enumerate(subsets_3):
    print(f"{_+1}: {tuple(subset.keys())}")
print()

	type(subsets_3): <class 'list'>
	len(subsets_3): 5
	type(subsets_3[0]): <class 'dict'>

1: ('VanDerPol0000',)
2: ('VanDerPol0000',)
3: ('VanDerPol0000',)
4: ('VanDerPol0000',)
5: ('VanDerPol0000',)



In [16]:
# Generate 2 examples of each size-n subset for n from 1 to 20;
# returns dict mapping n to list of datasets containing n worms
all_subsets = generate_all_subsets(
    combined_dataset,
    max_subsets_per_size=2,
    max_size=5,  # 20,
    as_assignment=True,
)
print(f"\ttype(all_subsets): {type(all_subsets)}")
print(f"\tlen(all_subsets): {len(all_subsets)}")
print()

	type(all_subsets): <class 'dict'>
	len(all_subsets): 5



In [17]:
# By using the `as_assignment=True` argument, we get a way to distribute worms to different
# experimental datasets such that we achieve the desired combined dataset size.
print("Distribution of worms in the first assignment:")
for size, subsets in all_subsets.items():
    print(f"size {size}: \t num. subsets: {len(subsets)} \t example: {subsets[0]}")
print()

# print("Distribution of worms in the the second assignment:")
# for size, subsets in all_subsets.items():
#     print(f"size {size}: \t num. subsets: {len(subsets)} \t example: {subsets[1]}")
# print()

Distribution of worms in the first assignment:
size 1: 	 num. subsets: 2 	 example: {'VanDerPol0000': 1}
size 2: 	 num. subsets: 2 	 example: {'VanDerPol0000': 2}
size 3: 	 num. subsets: 2 	 example: {'VanDerPol0000': 3}
size 4: 	 num. subsets: 2 	 example: {'VanDerPol0000': 4}
size 5: 	 num. subsets: 2 	 example: {'VanDerPol0000': 5}



In [18]:
# Do one assignment for all possible data subset sizes
all_subsets = generate_all_subsets(
    combined_dataset,
    max_subsets_per_size=1,
    max_size=None,
    as_assignment=True,
)
print(f"\ttype(all_subsets): {type(all_subsets)}")
print(f"\tlen(all_subsets): {len(all_subsets)}")
print()
print("Distribution of worms in assignment:")
for size, subsets in all_subsets.items():
    print(f"size {size}: \t num. subsets: {len(subsets)} \t example: {subsets[0]}")
print()

	type(all_subsets): <class 'dict'>
	len(all_subsets): 6

Distribution of worms in assignment:
size 1: 	 num. subsets: 1 	 example: {'VanDerPol0000': 1}
size 2: 	 num. subsets: 1 	 example: {'VanDerPol0000': 2}
size 3: 	 num. subsets: 1 	 example: {'VanDerPol0000': 3}
size 4: 	 num. subsets: 1 	 example: {'VanDerPol0000': 4}
size 5: 	 num. subsets: 1 	 example: {'VanDerPol0000': 5}
size 6: 	 num. subsets: 1 	 example: {'VanDerPol0000': 6}



In [42]:
# Example of how to get the Hydra config string
hydra_str = ""
for size, subsets in all_subsets.items():
    # removing quotes is important as hydra config parser treats parameters as a string
    _str = str(subsets[0]).replace("'", "")
    # need to remive space after colon (:) as hydra config parser throws an error otherwise
    _str = _str.replace(": ", ":")
    hydra_str += f"{_str},"
hydra_str = hydra_str[:-1]
print(f"{hydra_str}")

{VanDerPol0000:1},{VanDerPol0000:2},{VanDerPol0000:3},{VanDerPol0000:4},{VanDerPol0000:5},{VanDerPol0000:6}
